# Path Planning Interview Question

One challenge for our indycar is to generate a trajectory that avoids crashing into competing vehicles. Historically, we've represented this path using cubic splines. 


A cubic spline is a curve formed from a sequence of polynomial segments

 <img src="img/cubic_spline.png" width="500"/>


Each $i$th segment of the spline has a cubic equation,

$$y_i(t) = a_i (t-x_i)^3 + b_i (t-x_i)^2 + c_i (t-x_i) + d_i \; \mathrm{for} \; t \in [x_i, x_{i+1})$$


To simplify things, we're just going to work with one dimension, but can easily extend this to a 2D path like that of our car. 

We will constraint a few properties to make the curve useful to interpolate a path for our race car.

1. $y_i(x_i) = y_i$ and $y_i(x_{i+1}) = y_{i+1}$ for $i = 1, 2, ..., n-1$
2. $y_i'(x_i) = y_{i+1}'(x_{i+1})$ for $i = 1, 2, ..., n-2$
3. $y_i''(x_i) = y_{i+1}''(x_{i+1})$ for $i = 1, 2, ..., n-2$

Property 1 requires the curve to pass through all of the data points we specify. Properties 2 and 3 force the slope and curvature of the polynomial segments to be equal where they join. 

This gives us $3n - 3$ unknowns and $3n-5$ conditions. We'll impose the free boundary condition, $y_1''(x_1) = y_{n-1}'(x_n) = 0$, to add two more conditions and make the splines we calculate unique. 

In matrix form, we get

$$
\begin{bmatrix}
1 && 0 && 0 \\
\delta_1 && 2(\delta_1 + \delta_2) && \delta_2 && \ddots \\
0 && \delta_2 && 2(\delta_2 + delta_3) \\
&& \ddots && \ddots && \ddots && \\ 
&& && && \delta_{n-2} && 2 (\delta_{n-2} + \delta_{n-2}) && \delta_{n-1} \\
&& && && 0 && 0 && 1
\end{bmatrix}
\begin{bmatrix}
c_1 \\
\vdots \\
c_n
\end{bmatrix}
=
\begin{bmatrix}
0 \\
3 (\frac{\Delta_2}{\delta_2} - \frac{\Delta_1}{\delta_1}) \\
\vdots \\
3 (\frac{\Delta_{n-1}}{\delta_{n-1}} - \frac{\Delta_{n-2}}{\delta_{n-2}}) \\
0
\end{bmatrix}
$$

$$d_i = \frac{c_{i+1} - c_i}{3\delta_i}$$

$$b_i = \frac{\Delta_i}{\delta_i} - \frac{\delta_i}{3} (2c_i + c_{i+1}) $$

where $\delta_i = x_{i+1} - x_i$ and $\Delta_i = y_{i+1} - y_i$



## The Problem




In [ ]:
import numpy as np

def cubic_spline(x, y, tol=1e-100):
    """
    Interpolate using natural cubic splines.
    
    Generates a strictly diagonal dominant matrix then solves.
    
    Returns coefficients:
    b, coefficient of x of degree 1
    c, coefficient of x of degree 2
    d, coefficient of x of degree 3
    """ 
    
    x = np.array(x)
    y = np.array(y)
    
    # check if sorted
    if np.any(np.diff(x) < 0):
        idx = np.argsort(x)
        x = x[idx]
        y = y[idx]
        
    size = len(x)
    delta_x = np.diff(x)
    delta_y = np.diff(y)
    
    # Initialize to solve Ac = k
    A = np.zeros(shape = (size,size))
    k = np.zeros(shape=(size,1))
    A[0,0] = 1
    A[-1,-1] = 1
    
    for i in range(1,size-1):
        
        A[i, i-1] = delta_x[i-1]
        A[i, i+1] = delta_x[i]
        A[i,i] = 2*(delta_x[i-1]+delta_x[i])
        
        k[i,0] = 3*(delta_y[i]/delta_x[i] - delta_y[i-1]/delta_x[i-1])
    
    # Solves for c in Ac = k
    c = np.linalg.solve(A, k)
    
    # Solves for d and b
    d = np.zeros(shape = (size-1,1))
    b = np.zeros(shape = (size-1,1))
    for i in range(0,len(d)):
        d[i] = (c[i+1] - c[i]) / (3*delta_x[i])
        b[i] = (delta_y[i]/delta_x[i]) - (delta_x[i]/3)*(2*c[i] + c[i+1])    
    
    return x, y, b.squeeze(), c.squeeze(), d.squeeze()
    
    
def evaluate_spline(x, y, b, c, d, t):
    
    if t < x[0] or t > x[-1]:
        raise Exception("Can't extrapolate")

    # Index of segment to use
    idx = np.argmax(x > t) - 1
            
    dx = t - x[idx]
    value = y[idx] + b[idx]*dx + c[idx]*dx**2 + d[idx]*dx**3
    return value

def sample_spline(x, y, b, c, d, num=500):
    
    interp_y = np.zeros((num,))
    t = np.linspace(x[0], x[-1], num=num, endpoint=False)
    
    for i, t_i in enumerate(t):
        interp_y[i] = evaluate_spline(x, y, b, c, d, t_i)
        
    return t, interp_y
        
    

In [ ]:
import random
import matplotlib.pyplot as plt

key_points= np.array([[random.random()*3.0,random.random()*2.0-1.0] for i in range(5)])

plt.plot(key_points[:, 0], key_points[:, 1], "*b")

x, y, b, c, d = cubic_spline(key_points[:, 0], key_points[:, 1])

t, interp_y = sample_spline(x, y, b, c, d)


plt.plot(t, interp_y)

